In [19]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar
from sklearn.metrics import roc_auc_score, accuracy_score

In [2]:
tf.reset_default_graph()

In [10]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

<h1>Define Hyper-perparmeter</h1>

In [3]:
num_classes = 10
num_features = 3072
num_output = 10
num_layers_0 = 1024
num_layers_1 = 256
starter_learning_rate = 0.001
regularizer_rate = 0.1
batch_size = 128
epochs = 10
dropout_prob = 0.6
training_accuracy = []
training_loss = []
testing_accuracy = []

In [13]:
#### EXAMPLE, FEEL FREE TO CHANGE #####
valid_features, valid_labels = pickle.load(open('vali_data.pickle','rb'))
training_features,training_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))

<h1>Define Placeholder</h1>

In [4]:
# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_y = tf.placeholder('float32',shape = (None,num_classes),name="input_Y")
## for dropout layer
keep_prob = tf.placeholder(tf.float32)


In [5]:
## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_output], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_output]))

<h1>Define Neural Network Architecture</h1>

In [7]:
## Initializing weigths and biases
hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0) + bias_0)
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0_0,weights_1) + bias_1)
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
predicted_y = tf.sigmoid(tf.matmul(hidden_output_1_1,weights_2) + bias_2)

<h1>Define cost and optimization</h1>

In [8]:
## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_y,labels=input_y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(weights_0)) + tf.reduce_sum(tf.square(weights_1)) \
                           + tf.reduce_sum(tf.square(weights_2)))
## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2,
                                                                         bias_0,bias_1,bias_2])
correct_prediction = tf.equal(tf.argmax(input_y,1), tf.argmax(predicted_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for epoch in range(epochs): 
        start_time = time.time()
        for index in range(0,50000,batch_size):
            batch_x, batch_y = next_batch(batch_size, training_features, training_labels)
            print(batch_x.shape)
            print(batch_y.shape)
            s.run(optimizer, feed_dict={input_X:batch_x, input_y:batch_y, keep_prob:dropout_prob})
        use_time = time.time() - start_time
        training_accuracy.append(s.run(accuracy, feed_dict= {input_X:batch_x, 
                                                             input_y:batch_y,keep_prob:1}))
        training_loss.append(s.run(loss, {input_X: batch_x, 
                                          input_y: batch_y,keep_prob:1}))

        # Evaluation of model
        testing_accuracy.append(accuracy_score(vali_labels.argmax(1), 
                               s.run(predicted_y, {input_X: vali_features, keep_prob:1}).argmax(1)))
        print("Epoch:{0}, Train time: {1:.3f} Train loss: {2:.2f} Train acc: {3:.3f}, \
        Test acc:{4:.3f}".format(epoch, use_time, training_loss[epoch], training_accuracy[epoch], \
                                 validation_accuracy[epoch]))
    test_accuracy = accuracy_score(test_labels.argmax(1), s.run(predicted_y, {input_X: test_features, keep_prob:1}).argmax(1))
    print('testing accuracy is: ', test_accuracy)

ResourceExhaustedError: OOM when allocating tensor of shape [1024,256] and type float
	 [[Node: Variable_2/Adam/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [1024,256] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Variable_2/Adam/Initializer/zeros', defined at:
  File "/home/admin-cmmb/anaconda3/envs/ee596/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/admin-cmmb/anaconda3/envs/ee596/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/admin-cmmb/anaconda3/envs/ee596/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/admin-cmmb/anaconda3/envs/ee596/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/admin-cmmb/anaconda3/envs/ee596/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 50

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>